In [1]:
import torch
import pickle
import pandas as pd
import numpy as np
import transformers
import transformers
import ast
import json
import torch.nn as nn
from sklearn import metrics
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, matthews_corrcoef
#!python -m spacy download es_core_news_md

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


En la tarea de clasificación de texto en español, nos enfrentamos a la falta de recursos como léxicos (ya que los utilizados para la clasificación en inglés estaban en ese idioma). Para abordar esto, decidimos aprovechar las anotaciones a nivel de span disponibles en el dataset de la competición, donde se ofrecía información sobre elementos narrativos clave, como agentes y víctimas, que nos permitiría enriquecer el modelo de clasificación. Aunque realizamos un aumento de datos utilizando traducciones (primero al alemán y luego al español para obtener los mismos textos con palabras o expresiones cambiadas en la traducción), decidimos no usar este enfoque debido a que la traducción a veces resultó en textos incoherentes y, además, los nuevos textos no contaban con las anotaciones de span que íbamos a utilizar.

Optamos por usar DistilBERT, una versión compacta de BERT, debido a las limitaciones de recursos computacionales en Google Collab. Como los modelos de transformers solo aceptan vectores, decidimos incorporar los embeddings generados por SpaCy para las anotaciones de span (utilizamos SpaCy porque ofrecía una opción de modelo preentrenado en español: es_core_news_md) y concatenarlos con los embeddings de DistilBERT, permitiendo al modelo capturar relaciones más profundas dentro de los textos

# Carga y preprocesado de datos (Embeddings)

In [ ]:
df= pd.read_csv('data/train/dataset_es_train_augmented.csv')
df= df.iloc[:4000]
df = df[df['annotations'].notna()]  # Eliminar filas con NaN en la columna 'annotations'
df['annotations'] = df['annotations'].apply(ast.literal_eval)

In [ ]:
# Función para obtener el embedding de las anotaciones con SpaCy
import spacy

def get_spacy_embedding(annotations, spacy_model):
    embeddings = []
    for annotation in annotations:
        span_text = annotation['span_text']  # Texto de la anotación
        # Obtener el vector del span utilizando el modelo de SpaCy
        span_embedding = spacy_model(span_text).vector
        embeddings.append(span_embedding)

    # Si hay múltiples anotaciones, calcular el promedio de sus embeddings
    if len(embeddings) > 0:
        return torch.tensor(embeddings).mean(dim=0) 
    else:
        # Si no hay anotaciones, devolver un vector de ceros 
        return torch.zeros(spacy_model.vocab.vectors_length)

# Modelo en español
spacy_model = spacy.load("es_core_news_md")

# Lista para almacenar los embeddings de las anotaciones
spacy_embeddings = []


for index, row in df.iterrows():
    annotations = row["annotations"] 
    spacy_embed = get_spacy_embedding(annotations, spacy_model)
    spacy_embeddings.append(spacy_embed.numpy())

df["spacy_embedding"] = spacy_embeddings

print(df.head())


# Se guarda en formato pickle ya que en formato csv se producía un error al leer los embeddings
with open('data/train/dataset_es_train_completed.pkl', 'wb') as f:
    pickle.dump(df, f)

# Data 

In [2]:
# Datos con los embeddings
with open('data/train/dataset_es_train_completed.pkl', 'rb') as f:
    df = pickle.load(f)

print(type(df['spacy_embedding'].iloc[0])) 


<class 'numpy.ndarray'>


In [3]:
df= df[['id','text','category','spacy_embedding']]
df.head()

,id,text,category,spacy_embedding
0,2807,fallo en matrix hoy el señor joan ramón laport...,CRITICAL,"[0.9861435, 1.1332557, 1.0178244, -1.82641, 0...."
1,3054,siento ya tdas las vacunas vienen contaminadas...,CRITICAL,"[0.529693, -0.20333749, -2.8372486, 0.68856, 1..."
2,268,veo que curiosamente te autoproclamados interl...,CONSPIRACY,"[0.9124315, 2.0321434, -0.5982486, -1.0164008,..."
3,2669,documental vacunas una inyección en la oscurid...,CRITICAL,"[-0.7523574, -1.9963984, 0.43657142, -0.377231..."
4,3205,una sugerencia para los que se han vacunado y ...,CONSPIRACY,"[3.147237, 2.316337, -1.094331, -0.76536936, 2..."


In [4]:
# Transformar clases categóricas a número: clase 1 CRITICAL, clase 0 CONSPIRACY
df['class'] = df['category'].apply(lambda x: 1 if x == 'CRITICAL' else 0)
df

,id,text,category,spacy_embedding,class
0,2807,fallo en matrix hoy el señor joan ramón laport...,CRITICAL,"[0.9861435, 1.1332557, 1.0178244, -1.82641, 0....",1
1,3054,siento ya tdas las vacunas vienen contaminadas...,CRITICAL,"[0.529693, -0.20333749, -2.8372486, 0.68856, 1...",1
2,268,veo que curiosamente te autoproclamados interl...,CONSPIRACY,"[0.9124315, 2.0321434, -0.5982486, -1.0164008,...",0
3,2669,documental vacunas una inyección en la oscurid...,CRITICAL,"[-0.7523574, -1.9963984, 0.43657142, -0.377231...",1
4,3205,una sugerencia para los que se han vacunado y ...,CONSPIRACY,"[3.147237, 2.316337, -1.094331, -0.76536936, 2...",0
...,...,...,...,...,...
3995,1056,dr . robert malone . co inventor de las tecnol...,CRITICAL,"[1.4721507, -0.20181565, 0.16380507, -0.928432...",1
3996,861,una pregunta la vacuna también provoca hipotir...,CRITICAL,"[0.92393684, 0.4082314, 0.42118683, -1.2820561...",1
3997,5248,eric clapton el famoso guitarrista cuenta como...,CRITICAL,"[-0.4229155, 1.1633127, -1.2490486, -0.2680207...",1
3998,1328,"no es médico , no es científico , no es biólog...",CONSPIRACY,"[1.5191575, 0.9244397, -0.20330362, -1.5485926...",0


# Parámetros

In [5]:
# Misma configuración que en la tarea de clasificación en inglés
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [6]:
# Preparar el dataset teniendo en cuenta los embeddings
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, spacy_embeddings, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['text']
        self.targets = self.data['class']
        self.spacy_embeddings = spacy_embeddings  
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        # Tokenizar el texto con BERT o DistilBERT
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        ids = inputs['input_ids'].squeeze()
        mask = inputs['attention_mask'].squeeze()

        # Obtener el embedding de SpaCy para el índice correspondiente
        spacy_embedding = self.spacy_embeddings[index]

        return {
            'ids': ids,
            'mask': mask,
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
            'spacy_embedding': torch.tensor(spacy_embedding, dtype=torch.float)  # Añadir los embeddings de SpaCy
        }


# Modelo

In [7]:
# Concatenamos ambos vectores, en primer lugar los embeddings del encoder distilbert (el token CLS que 
# contiene toda la información del textoy los embeddings de spacy

class DistilBERTClass(torch.nn.Module):
    def __init__(self, spacy_embed_size=300):
        super(DistilBERTClass, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
        self.dropout = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768 + spacy_embed_size, 1)  

    def forward(self, ids, mask, spacy_embed):
        # Pasar por DistilBERT
        distilbert_output = self.distilbert(ids, attention_mask=mask)
        distilbert_embedding = distilbert_output[0][:, 0, :] 

        # Concatenar los embeddings de DistilBERT con los de SpaCy
        combined_embedding = torch.cat((distilbert_embedding, spacy_embed), dim=1)

        # Aplicar Dropout
        combined_embedding = self.dropout(combined_embedding)

        # Pasar por la capa completamente conectada
        output = self.fc(combined_embedding).squeeze(1)

        return output


model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

# Fine Tuning (Cross Validation)

In [8]:
# Para entrenar y evaluar el modelo, se implementó validación cruzada (cross-validation) con 5 subconjuntos (k=5). 
# En cada uno, se entrenó el modelo usando StratifiedKFold para asegurar que la distribución de clases fuera similar en 
# cada conjunto de entrenamiento y validación, dado que no se había realizado aumento de datos y las clases no estaban equilibradas.

def train(epoch, model, train_loader, optimizer, loss_fn, device):
    model.train()
    epoch_loss = [] #Pérdidas por época

    for _, data in enumerate(train_loader):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        spacy_embed = data['spacy_embedding'].to(device, dtype=torch.float)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids, mask, spacy_embed)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())  

        if _ % 500 == 0: 
            print(f'Epoch: {epoch}, Step: {_}, Loss: {loss.item()}')
    return epoch_loss


def validation(model, val_loader, device):
    model.eval()
    fin_targets = []
    fin_outputs = []

    with torch.no_grad():
        for data in val_loader:
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            spacy_embed = data['spacy_embedding'].to(device) 
            targets = data['targets'].to(device)
            outputs = model(ids, mask, spacy_embed)
            outputs = torch.sigmoid(outputs) 
            fin_targets.extend(targets.cpu().numpy())
            fin_outputs.extend(outputs.cpu().numpy())
    return fin_outputs, fin_targets


def evaluate_metrics(outputs, targets):
    # Convertir las salidas a 0 o 1 (clases predichas) basadas en el umbral 0.5
    outputs = [1 if x > 0.5 else 0 for x in outputs]

    # Métricas
    accuracy = accuracy_score(targets, outputs)
    precision, recall, f1, _ = precision_recall_fscore_support(targets, outputs, average='binary')
    mcc = matthews_corrcoef(targets, outputs)  # Calcular MCC
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'mcc': mcc
    }
def cross_validate_model(model, dataframe, tokenizer, spacy_embeddings, epochs=3, batch_size=16, k_folds=5, max_len=128):
    kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    metrics_list = []
    all_train_losses = []  

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataframe, dataframe['class'])):
        print(f"\nFold {fold + 1}/{k_folds}")
        train_df = dataframe.iloc[train_idx].reset_index(drop=True)
        val_df = dataframe.iloc[val_idx].reset_index(drop=True)

        train_set = CustomDataset(train_df, tokenizer, spacy_embeddings=train_df['spacy_embedding'], max_len=max_len)
        val_set = CustomDataset(val_df, tokenizer, spacy_embeddings=val_df['spacy_embedding'], max_len=max_len)

        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
        
        #Al tener una tarea de clasificación binaria se ha utilizado binary crossentropy como función de pérdidas
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
        loss_fn = torch.nn.BCEWithLogitsLoss()

        fold_train_losses = []  

        for epoch in range(epochs):
            epoch_loss = train(epoch, model, train_loader, optimizer, loss_fn, device)
            fold_train_losses.append(epoch_loss)

        all_train_losses.append(fold_train_losses) 

        outputs, targets = validation(model, val_loader, device)
        fold_metrics = evaluate_metrics(outputs, targets)
        metrics_list.append(fold_metrics)

    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.to_csv('metrics.csv', index=False)
    print('Cross-validation complete')

    with open('losses.json', 'w') as f:
        json.dump(all_train_losses, f)


In [9]:
cross_validate_model(model, df, tokenizer, 'metrics', epochs=3, batch_size=TRAIN_BATCH_SIZE, k_folds=5)


Fold 1/5
Epoch: 0, Step: 0, Loss: 0.6861395835876465
Epoch: 1, Step: 0, Loss: 0.586403489112854
Epoch: 2, Step: 0, Loss: 0.30674028396606445

Fold 2/5
Epoch: 0, Step: 0, Loss: 0.2893897294998169
Epoch: 1, Step: 0, Loss: 0.18789798021316528
Epoch: 2, Step: 0, Loss: 0.15846575796604156

Fold 3/5
Epoch: 0, Step: 0, Loss: 0.10516831278800964
Epoch: 1, Step: 0, Loss: 0.03810688853263855
Epoch: 2, Step: 0, Loss: 0.03411080688238144

Fold 4/5
Epoch: 0, Step: 0, Loss: 0.1148531511425972
Epoch: 1, Step: 0, Loss: 0.06887974590063095
Epoch: 2, Step: 0, Loss: 0.009534399956464767

Fold 5/5
Epoch: 0, Step: 0, Loss: 0.02601243183016777
Epoch: 1, Step: 0, Loss: 0.012072348967194557
Epoch: 2, Step: 0, Loss: 0.026457849889993668
Cross-validation complete


In [ ]:
#torch.save(model_cpu.state_dict(), 'spanish_model.pth')

# Test

In [ ]:
# Para los datos de test se calcularon los correspondientes embeddings de la columna annotations
# Debido a que el modelo fue ajustado usando éstos se calcularon para su posible uso
df= pd.read_csv('data/test/dataset_es_test_completed.csv')
df['annotations'] = df['annotations'].apply(ast.literal_eval)

for index, row in df.iterrows():
    annotations = row["annotations"] 
    spacy_embed = get_spacy_embedding(annotations, spacy_model)
    spacy_embeddings.append(spacy_embed.numpy())

df["spacy_embedding"] = spacy_embeddings
print(df.head())


with open('data/train/dataset_es_test_completed.pkl', 'wb') as f:
    pickle.dump(df, f)

In [10]:
#Abrir dataset ya preprocesado por facilidad
with open('data/test/dataset_es_test_completed.pkl', 'rb') as f:
    df = pickle.load(f)

#verificar formato de embeddings
#print(type(df['spacy_embedding'].iloc[0])) 
df['class'] = df['category'].apply(lambda x: 1 if x == 'CRITICAL' else 0)
df.head()

,Unnamed: 0,id,text,category,annotations,spacy_tokens,spacy_embedding,class
0,0,1902,con referencia a los datos oficiales del de ma...,CRITICAL,[{'span_text': 'los Totalmente Vacunados ahora...,WyJDb24iLCAicmVmZXJlbmNpYSIsICJhIiwgImxvcyIsIC...,"[2.4959738, 0.15475102, -0.057252802, -0.44742...",1
1,1,2896,"atención , padres de alumnos en edad escolar ....",CRITICAL,[{'span_text': 'El prestigioso abogado valenci...,WyJBVEVOQ0lcdTAwZDNOIiwgIiwiLCAiUEFEUkVTIiwgIk...,"[-1.1329516, -1.2203003, 0.9488069, -0.8421083...",1
2,2,2509,la directora de los cdc admite que no hubo pan...,CONSPIRACY,"[{'span_text': 'La directora de los CDC', 'cat...",WyJMYSIsICJkaXJlY3RvcmEiLCAiZGUiLCAibG9zIiwgIk...,"[2.55884, 0.7957912, 0.70012844, -2.4257998, 0...",0
3,3,4037,. msn . com es mx health noticias medicas oms ...,CRITICAL,"[{'span_text': 'personas - de - riesgo', 'cate...",WyJodHRwcyIsICI6Ly8iLCAid3d3IiwgIi4iLCAibXNuIi...,"[0.39429554, -0.86550504, 0.16315424, -2.21528...",1
4,4,3660,multitudinarias manifestaciones en toda españa...,CRITICAL,"[{'span_text': 'niños', 'category': 'VICTIM', ...",WyIgIiwgIk11bHRpdHVkaW5hcmlhcyIsICJtYW5pZmVzdG...,"[2.3765001, 1.15517, 0.396785, -1.03143, 3.221...",1


In [11]:
model.eval()

DistilBERTClass(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

In [14]:
from sklearn.metrics import classification_report

def predict(text, model, tokenizer, spacy_embed=None, max_len=512):
    # Preparar datos para inferencia
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt')
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    if spacy_embed is not None:
        if isinstance(spacy_embed, np.ndarray):
            spacy_embed = torch.tensor(spacy_embed).float() 
        spacy_embed = spacy_embed.unsqueeze(0).to(device) 
    
    # Hacer la predicción
    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask, spacy_embed)  
        probability = torch.sigmoid(output).cpu().numpy()  
        prediction = (probability >= 0.5).astype(int)  

    return prediction, probability


def evaluate_model(model, tokenizer, test_df, spacy_embed_col=None, filename="metrics.json"):
    all_predictions = []
    all_labels = []

    for index, row in test_df.iterrows():
        text = row['text']
        label = row['class']
        
        # Si se requiere embedding de spaCy
        if spacy_embed_col:
            spacy_embed = row[spacy_embed_col]
        else:
            spacy_embed = torch.zeros(300)  

        # Realizar la predicción
        prediction, probability = predict(text, model, tokenizer, spacy_embed=spacy_embed)
        
        all_predictions.extend(prediction)
        all_labels.append(label)

    report = classification_report(all_labels, all_predictions, digits=5, output_dict=True)
    mcc = matthews_corrcoef(all_labels, all_predictions)

    # Guardar las métricas
    metrics = {
        "classification_report": report,
        "mcc": mcc
    }

    with open(filename, 'w') as f:
        json.dump(metrics, f, indent=4)

    print("Classification Report:\n", report)
    print(f"MCC: {mcc:.4f}")

    return report, mcc

report, mcc = evaluate_model(model, tokenizer, df, spacy_embed_col='spacy_embedding', filename="metrics.json")


Classification Report:
 {'0': {'precision': 0.8235294117647058, 'recall': 0.5737704918032787, 'f1-score': 0.6763285024154589, 'support': 366.0}, '1': {'precision': 0.7906040268456376, 'recall': 0.9290220820189274, 'f1-score': 0.8542422044960116, 'support': 634.0}, 'accuracy': 0.799, 'macro avg': {'precision': 0.8070667193051717, 'recall': 0.751396286911103, 'f1-score': 0.7652853534557353, 'support': 1000.0}, 'weighted avg': {'precision': 0.8026547177260166, 'recall': 0.799, 'f1-score': 0.7891257895345294, 'support': 1000.0}}
MCC: 0.5557
